In [ ]:
!pip install rsmtool

In [ ]:
import numpy as np
import pandas as pd
from rsmtool.utils.metrics import quadratic_weighted_kappa, difference_of_standardized_means, standardized_mean_difference
from collections import Counter
from keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense, Convolution1D, GlobalMaxPooling1D, concatenate, Dropout, BatchNormalization, Activation
from keras.models import Model
from keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('punkt')
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from rsmtool.utils.metrics import quadratic_weighted_kappa, difference_of_standardized_means, standardized_mean_difference
from rsmtool.fairness_utils import get_fairness_analyses
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.callbacks import EarlyStopping
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
fp1=open("","r")
glove_emb={}
for line in fp1:
	temp=line.split(" ")
	glove_emb[temp[0]]=np.asarray([float(i) for i in temp[1:]])

In [ ]:
def load_data(path):
  prompt_1 = pd.read_csv(path+'Prompt_1.csv')
  prompt_2 = pd.read_csv(path+'Prompt_2.csv')
  prompt_3 = pd.read_csv(path+'Prompt_3.csv')
  prompt_4 = pd.read_csv(path+'Prompt_4.csv')
  prompt_5 = pd.read_csv(path+'Prompt_5.csv')
  prompt_6 = pd.read_csv(path+'Prompt_6.csv')
  prompt_7 = pd.read_csv(path+'Prompt_7.csv')
  prompt_8 = pd.read_csv(path+'Prompt_8.csv')
  prompt_9 = pd.read_csv(path+'Prompt_9.csv')
  prompt_10 = pd.read_csv(path+'Prompt_10.csv')
  prompt_11 = pd.read_csv(path+'Prompt_11.csv')
  prompt_12 = pd.read_csv(path+'Prompt_12.csv')
  prompt_1_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_1.csv')
  prompt_2_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_2.csv')
  prompt_3_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_3.csv')
  prompt_4_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_4.csv')
  prompt_5_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_5.csv')
  prompt_6_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_6.csv')
  prompt_7_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_7.csv')
  prompt_8_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_8.csv')
  prompt_9_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_9.csv')
  prompt_10_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_10.csv')
  prompt_11_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_11.csv')
  prompt_12_score = pd.read_csv('/content/drive/MyDrive/AES/Rank_SVM/pseudo_12.csv')

  return [(prompt_1, prompt_1_score), (prompt_2, prompt_2_score),
          (prompt_3, prompt_3_score), (prompt_4, prompt_4_score),
          (prompt_5, prompt_5_score), (prompt_6, prompt_6_score),
          (prompt_7, prompt_7_score), (prompt_8, prompt_8_score),
          (prompt_9, prompt_9_score), (prompt_10, prompt_10_score),
          (prompt_11, prompt_11_score), (prompt_12, prompt_12_score)]

def covert_label(scores):
  range_min = 1
  range_max = 6
  scaled_scores = (scores-range_min)/(range_max-range_min)
  return scaled_scores

def get_training_samples(texts, scaled_scores):
  scaled_scores_temp = scaled_scores*9 + 1
  negative_index = np.where(scaled_scores_temp<=4)[0]
  positive_index = np.where(scaled_scores_temp>=8)[0]
  combined_array = np.concatenate((negative_index, positive_index))
  return texts[combined_array], scaled_scores[combined_array]

def get_features(texts):
  tokenizer=Tokenizer(num_words=4000, oov_token = "<unk>")
  tokenizer.fit_on_texts(texts)
  word_index=tokenizer.word_index
  essayset = []
  for text in texts:
    sentences = nltk.tokenize.sent_tokenize(text)
    essay = []
    essay = tokenizer.texts_to_sequences(sentences)
    essay1 = pad_sequences(essay, 40, padding='post')
    essayset.append(essay1)
  maxlen = max([len(x) for x in essayset])
  essayset = pad_sequences(essayset, maxlen, padding='post')
  return essayset, word_index

def split_data(data, fold):
    kfold = KFold(n_splits=fold, shuffle=False)
    results = []
    for train_index, test_index in kfold.split(data):
        results.append((train_index, test_index))
    return results

def accuracy_evaluation(y_pred, y_test):
    qwk = quadratic_weighted_kappa(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    pearson_score = pearsonr(y_test, y_pred).statistic
    return qwk, mae, pearson_score

def fairness_evaluation(y_pred, y_test, demo_attribute):
    df = pd.DataFrame({"True_Score":y_test, "Prediction_Score":y_pred, "Demo":demo_attribute})
    results = get_fairness_analyses(df, group="Demo", system_score_column="Prediction_Score", human_score_column="True_Score")[1].values()[3]
    population_y_true_observed_sd = np.std(y_test)
    population_y_true_observed_mn = np.mean(y_test)
    population_y_pred_sd = np.std(y_pred)
    population_y_pred_mn = np.mean(y_pred)
    y_test_demo_0 = y_test[np.where(demo_attribute==0)]
    y_test_demo_1 = y_test[np.where(demo_attribute==1)]
    y_pred_demo_0 = y_pred[np.where(demo_attribute==0)]
    y_pred_demo_1 = y_pred[np.where(demo_attribute==1)]
    SMD_0 = difference_of_standardized_means(y_test_demo_0, y_pred_demo_0, population_y_true_observed_mn, population_y_pred_mn, population_y_true_observed_sd, population_y_pred_sd)
    SMD_1 = difference_of_standardized_means(y_test_demo_1, y_pred_demo_1, population_y_true_observed_mn, population_y_pred_mn, population_y_true_observed_sd, population_y_pred_sd)
    diff_mae = mean_absolute_error(y_test_demo_1, y_pred_demo_1) - mean_absolute_error(y_test_demo_0, y_pred_demo_0)
    scores = pd.DataFrame({"SMD_0":[SMD_0], "SMD_1":[SMD_1], "diff_mae":[diff_mae]})
    return results, scores

In [ ]:
def model_essay(maxSenLen, vocab_size, embedding_dim, hidden_dim, dense_dim, word_index):
  embedding_matrix = np.zeros((vocab_size, embedding_dim))
  for word,i in word_index.items():
    if i >= vocab_size:
      continue
    if word in glove_emb:
      embedding_matrix[i]=glove_emb[word]
  input_words = Input(shape=(maxSenLen,), dtype='int32')
  embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True, mask_zero=True)(input_words)
  bi_lstm_layer = Bidirectional(LSTM(units=hidden_dim, return_sequences=False), merge_mode='concat')(embedding_layer)
  sentence_model = Model(inputs=input_words, outputs=bi_lstm_layer)
  input_essay = Input(shape=(None, maxSenLen), dtype='int32')
  essay_layer = TimeDistributed(sentence_model)(input_essay)
  essay_bilstm_layer = Bidirectional(LSTM(units=hidden_dim, return_sequences=False), merge_mode='concat')(essay_layer)
  bn_merge_layer2 = BatchNormalization()(essay_bilstm_layer)
  merge_dense_layer2 = Dense(dense_dim, activation='relu')(bn_merge_layer2)
  score_layer = Dense(1, activation='sigmoid', name='pred_score')(merge_dense_layer2)
  essay_model = Model(inputs=input_essay, outputs=score_layer)
  optimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=0, clipvalue=10)
  essay_model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=['mean_squared_error'])
  return essay_model

In [ ]:
def run_experiment(seed):
  df = pd.DataFrame(columns=["prompt", "fold", "quadratic_weighted_kappa", "mean_absolute_error", "pearson_correlation_coefficient",
                            "OSA_gender", "OSA_gender_p_value", "OSD_gender", "OSD_gender_p_value", "CSD_gender", "CSD_gender_p_value", "SMD_1_gender", "SMD_0_gender", "MAED_gender",
                            "OSA_Economically_disadvantaged", "OSA_Economically_disadvantaged_p_value", "OSD_Economically_disadvantaged", "OSD_Economically_disadvantaged_p_value", "CSD_Economically_disadvantaged", "CSD_Economically_disadvantaged_p_value", "SMD_1_Economically_disadvantaged", "SMD_0_Economically_disadvantaged", "MAED_Economically_disadvantaged",
                            "OSA_Disability", "OSA_Disability_p_value", "OSD_Disability", "OSD_Disability_p_value", "CSD_Disability", "CSD_Disability_p_value", "SMD_1_Disability", "SMD_0_Disability", "MAED_Disability",
                            "OSA_English_Language_Learner", "OSA_English_Language_Learner_p_value", "OSD_English_Language_Learner", "OSD_English_Language_Learner_p_value", "CSD_English_Language_Learner", "CSD_English_Language_Learner_p_value", "SMD_1_English_Language_Learner", "SMD_0_English_Language_Learner", "MAED_English_Language_Learner",
                            "OSA_Race", "OSA_Race_p_value", "OSD_Race", "OSD_Race_p_value", "CSD_Race", "CSD_Race_p_value", "SMD_1_Race", "SMD_0_Race", "MAED_Race"])
  prompts = load_data("")
  i = 0
  for prompt in prompts:
    print("Prompt"+str(i+1))
    X, word_index = get_features(prompt[0]["Text"].to_numpy())
    y = covert_label(prompt[1].to_numpy().reshape(-1))
    y_real = covert_label(prompt[0]["Overall"].to_numpy())

    X_train, y_train = get_training_samples(X, y)
    kfolds = split_data(prompt[0], 5)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

    earlystopping = EarlyStopping(monitor="val_loss", patience=5)
    model = model_essay(40, 4000, 50, 50, 50, word_index)
    hist = model.fit([X_train], y_train, batch_size=16, epochs=50, validation_data=([X_val], y_val), callbacks=[earlystopping])
    k = 0
    for kfold in kfolds:
      X_test = X[kfold[1]]
      y_test = y_real[kfold[1]]*5+1
      test_info = prompt[0].iloc[kfold[1]]
      y_pred = model.predict(X_test).reshape(-1)
      y_pred = y_pred*5+1
      qwk, mae, pearson_score = accuracy_evaluation(y_pred, y_test)
      print(str(qwk), str(mae), str(pearson_score))
      fairness_part1_Gender, fairness_part2_Gender = fairness_evaluation(y_pred, y_test, test_info['Gender'].to_numpy())
      fairness_part1_Economically_disadvantaged, fairness_part2_Economically_disadvantaged = fairness_evaluation(y_pred, y_test, test_info['Economically_disadvantaged'].to_numpy())
      fairness_part1_Disability, fairness_part2_Disability = fairness_evaluation(y_pred, y_test, test_info['Disability'].to_numpy())
      fairness_part1_English_Language_Learner, fairness_part2_English_Language_Learner = fairness_evaluation(y_pred, y_test, test_info['English_Language_Learner'].to_numpy())
      fairness_part1_Race, fairness_part2_Race = fairness_evaluation(y_pred, y_test, test_info['Race_Binary'].to_numpy())
      new_row = {"prompt" : i+1, "fold": k+1, "quadratic_weighted_kappa": qwk, "mean_absolute_error": mae, "pearson_correlation_coefficient": pearson_score,
                      "OSA_gender": fairness_part1_Gender['Overall score accuracy']['R2'],
                      "OSA_gender_p_value": fairness_part1_Gender['Overall score accuracy']['sig'],
                      "OSD_gender": fairness_part1_Gender['Overall score difference']['R2'],
                      "OSD_gender_p_value": fairness_part1_Gender['Overall score difference']['sig'],
                      "CSD_gender": fairness_part1_Gender['Conditional score difference']['R2'],
                      "CSD_gender_p_value": fairness_part1_Gender['Conditional score difference']['sig'],
                      "SMD_1_gender":fairness_part2_Gender['SMD_1'][0],
                      "SMD_0_gender":fairness_part2_Gender['SMD_0'][0],
                      "MAED_gender":fairness_part2_Gender['diff_mae'][0],
                      "OSA_Economically_disadvantaged": fairness_part1_Economically_disadvantaged['Overall score accuracy']['R2'],
                      "OSA_Economically_disadvantaged_p_value": fairness_part1_Economically_disadvantaged['Overall score accuracy']['sig'],
                      "OSD_Economically_disadvantaged": fairness_part1_Economically_disadvantaged['Overall score difference']['R2'],
                      "OSD_Economically_disadvantaged_p_value": fairness_part1_Economically_disadvantaged['Overall score difference']['sig'],
                      "CSD_Economically_disadvantaged": fairness_part1_Economically_disadvantaged['Conditional score difference']['R2'],
                      "CSD_Economically_disadvantaged_p_value": fairness_part1_Economically_disadvantaged['Conditional score difference']['sig'],
                      "SMD_1_Economically_disadvantaged":fairness_part2_Economically_disadvantaged['SMD_1'][0],
                      "SMD_0_Economically_disadvantaged":fairness_part2_Economically_disadvantaged['SMD_0'][0],
                      "MAED_Economically_disadvantaged":fairness_part2_Economically_disadvantaged['diff_mae'][0],
                      "OSA_Disability": fairness_part1_Disability['Overall score accuracy']['R2'],
                      "OSA_Disability_p_value": fairness_part1_Disability['Overall score accuracy']['sig'],
                      "OSD_Disability": fairness_part1_Disability['Overall score difference']['R2'],
                      "OSD_Disability_p_value": fairness_part1_Disability['Overall score difference']['sig'],
                      "CSD_Disability": fairness_part1_Disability['Conditional score difference']['R2'],
                      "CSD_Disability_p_value": fairness_part1_Disability['Conditional score difference']['sig'],
                      "SMD_1_Disability":fairness_part2_Disability['SMD_1'][0],
                      "SMD_0_Disability":fairness_part2_Disability['SMD_0'][0],
                      "MAED_Disability":fairness_part2_Disability['diff_mae'][0],
                      "OSA_English_Language_Learner": fairness_part1_English_Language_Learner['Overall score accuracy']['R2'],
                      "OSA_English_Language_Learner_p_value": fairness_part1_English_Language_Learner['Overall score accuracy']['sig'],
                      "OSD_English_Language_Learner": fairness_part1_English_Language_Learner['Overall score difference']['R2'],
                      "OSD_English_Language_Learner_p_value": fairness_part1_English_Language_Learner['Overall score difference']['sig'],
                      "CSD_English_Language_Learner": fairness_part1_English_Language_Learner['Conditional score difference']['R2'],
                      "CSD_English_Language_Learner_p_value": fairness_part1_English_Language_Learner['Conditional score difference']['sig'],
                      "SMD_1_English_Language_Learner":fairness_part2_English_Language_Learner['SMD_1'][0],
                      "SMD_0_English_Language_Learner":fairness_part2_English_Language_Learner['SMD_0'][0],
                      "MAED_English_Language_Learner":fairness_part2_English_Language_Learner['diff_mae'][0],
                      "OSA_Race": fairness_part1_Race['Overall score accuracy']['R2'],
                      "OSA_Race_p_value": fairness_part1_Race['Overall score accuracy']['sig'],
                      "OSD_Race": fairness_part1_Race['Overall score difference']['R2'],
                      "OSD_Race_p_value": fairness_part1_Race['Overall score difference']['sig'],
                      "CSD_Race": fairness_part1_Race['Conditional score difference']['R2'],
                      "CSD_Race_p_value": fairness_part1_Race['Conditional score difference']['sig'],
                      "SMD_1_Race":fairness_part2_Race['SMD_1'][0],
                      "SMD_0_Race":fairness_part2_Race['SMD_0'][0],
                      "MAED_Race":fairness_part2_Race['diff_mae'][0]}
      df = df.append(new_row, ignore_index=True)
      k += 1
    df.to_csv('', index=False)
    i += 1
  return df

In [ ]:
run_experiment(0)